# Example: Meshing and creating .stl model from .json annotions

For the purpose of illustrating these scripts, annotions(.json files) of sub-01 were placed in annots/ folder. For default, the output would be saved in outputs/ folder. These codes were tested and executed on Ubuntu 23.04.

## Step up envirionment

Creating conda environment and install relied modules.
```
conda create -n Smgenv python=3.11
conda activate Smgenv
pip install bpy
pip install pyvista[all]
pip install pandas
pip install scipy
```

## Import required modules

In [24]:
import sys
import os
import bpy
import pyvista as pv
from utils.blender_tools import initialize, clearMesh
from utils.AnnotationImport import importPoints
from utils.MeshBuilding import getMeshSplines, bridgeLoopMeshes, reloc_entrypoint, smoothingLines, getnsaveMeshOpenSplines

# data import
annotation_base_path = 'annots'
# model save
modelsave_path = 'outputs'
# subject number
sub_num = 'sub-03'

## Initialize and import annotations

In [25]:
# blender initialization
initialize()
scene = bpy.context.scene
scene.unit_settings.system = 'METRIC'
scene.unit_settings.scale_length = 0.001

# creating sub path in outputs
singlesub_path=os.path.join(modelsave_path,sub_num)
os.makedirs(singlesub_path,exist_ok=True)

# target nerveroots and their radius
SEG = ["L1", "L2", "L3", "L4", "L5", "S1", "S2"]
Rs = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]

# import original annation points
dura, cord, lines = importPoints(annotation_base_path, SEG, sub_num)

print(f'imported dura for {dura.__len__()} loops')
print(f'imported cord for {cord.__len__()} loops')
print(f'imported spinal cord roots for {lines.__len__()} lines')

imported dura for 80 loops
imported cord for 66 loops
imported spinal cord roots for 14 lines


## Build and visualise dura section

In [26]:
# dura mesh building 
getMeshSplines(dura, "Dura")
bridgeLoopMeshes("Dura", sub_num, singlesub_path)
clearMesh()

# visualise the output using pyvista
plotter = pv.Plotter()
dura_mesh = pv.read(os.path.join(singlesub_path,f"{sub_num}_Dura.stl"))
plotter.add_mesh(dura_mesh)
plotter.show(jupyter_backend='html')

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

## Build and visualise cord section

In [27]:
# cord mesh building
getMeshSplines(cord, "Cord")
bridgeLoopMeshes("Cord", sub_num, singlesub_path)
clearMesh()

# visualise the output using pyvista
plotter = pv.Plotter()
cord_mesh = pv.read(os.path.join(singlesub_path,f"{sub_num}_Cord.stl"))
plotter.add_mesh(cord_mesh)
plotter.show(jupyter_backend='html')

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

## Build and visualise nerveroots section

In [28]:
# nerveroots mesh building
cordstl_path = os.path.join(singlesub_path,f"{sub_num}_Cord.stl")
lines_relocated = reloc_entrypoint(cordstl_path, lines)
lines_smoothed = smoothingLines(lines_relocated, n_interpolate = 100, s=10)
getnsaveMeshOpenSplines(lines_smoothed, SEG, Rs, sub_num,singlesub_path)

# visualise the output using pyvista
plotter = pv.Plotter()
roots_mesh = pv.read(os.path.join(singlesub_path,f"{sub_num}_Nerveroots.stl"))
plotter.add_mesh(roots_mesh)
plotter.show(jupyter_backend='html')

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

## Overall illustration

In [29]:
plotter = pv.Plotter()
# setting varied opacity to show inner sections
plotter.add_mesh(roots_mesh,opacity=1)
plotter.add_mesh(cord_mesh,opacity=1)
plotter.add_mesh(dura_mesh,opacity=0.3)
plotter.show(jupyter_backend='html')

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

## Final adjustment (if needed)

Considering possible bias caused by manual labeling and interpolation, sometimes the edge of dura may have intersections with nerve roots before the nerve roots through the foramen intervertebral. In such scenario, the model can be imported into blender and adjusted using sculpting function as shown in follows.

In [30]:
# preparation for manual sculpting in blender
bpy.ops.import_mesh.stl(filepath=os.path.join(singlesub_path,f'{sub_num}_Dura.stl'))
bpy.ops.import_mesh.stl(filepath=os.path.join(singlesub_path,f'{sub_num}_Cord.stl'))

Import finished in 0.8820 sec.
Import finished in 0.4792 sec.


{'FINISHED'}